In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet("./Mistral-7B-Instruct-v0.2_rv4-ld0-bs2-p0-nta0-e1_engpeft-pipbasic-t0.2-topp0.95_executed.parquet.gzip")
df

,row,last_executed_step,to_be_executed_step,translated_prompt,status,has_error,input,target_template,target_raw,execution,executed_query
2418,"""Can you find me all the dead authors who have...",LLMTranslator,,SELECT ?item ?itemLabel WHERE {\n ?item w...,,False,"""Can you find me all the dead authors who have...",SELECT DISTINCT ?person ?personLabel ?personDe...,SELECT DISTINCT ?person ?personLabel ?personDe...,"[{'item': {'type': 'uri', 'value': 'http://www...",SELECT ?item ?itemLabel WHERE {\n ?item w...
2419,"""Can you find me all the mayors who are any ki...",LLMTranslator,,SELECT ?item ?itemLabel WHERE {\n ?item w...,,False,"""Can you find me all the mayors who are any ki...",SELECT ?image ?speciesLabel ?mayorLabel ?place...,SELECT ?image ?speciesLabel ?mayorLabel ?place...,"[{'item': {'type': 'uri', 'value': 'http://www...",SELECT ?item ?itemLabel WHERE {\n ?item w...
2420,"""Can you retrieve a list of distinct instituti...",LLMTranslator,,SELECT ?item ?itemLabel WHERE {\n ?item w...,,False,"""Can you retrieve a list of distinct instituti...",select distinct ?item ?itemLabel (sample(?logo...,select distinct ?item ?itemLabel (sample(?logo...,"[{'item': {'type': 'uri', 'value': 'http://www...",SELECT ?item ?itemLabel WHERE {\n ?item w...
2421,"""Can you retrieve a map of the highest points ...",LLMTranslator,,SELECT ?item ?itemLabel WHERE {\n ?item w...,,False,"""Can you retrieve a map of the highest points ...",SELECT DISTINCT ?item ?itemLabel ?highestPoint...,SELECT DISTINCT ?item ?itemLabel ?highestPoint...,"[{'item': {'type': 'uri', 'value': 'http://www...",SELECT ?item ?itemLabel WHERE {\n ?item w...
2422,"""Can you retrieve information about long-runni...",LLMTranslator,,SELECT ?item ?itemLabel WHERE {\n ?item w...,,False,"""Can you retrieve information about long-runni...",SELECT ?family ?familyLabel (MAX(?age) AS ?age...,SELECT ?family ?familyLabel (MAX(?age) AS ?age...,"[{'item': {'type': 'uri', 'value': 'http://www...",SELECT ?item ?itemLabel WHERE {\n ?item w...
...,...,...,...,...,...,...,...,...,...,...,...
2840,"""Can you retrieve a list of movies directed by...",LLMTranslator,,SELECT ?item ?itemLabel WHERE {\n ?item w...,,False,"""Can you retrieve a list of movies directed by...",SELECT ?film ?filmLabel ?seriesLabel ?duration...,SELECT ?film ?filmLabel ?seriesLabel ?duration...,"[{'item': {'type': 'uri', 'value': 'http://www...",SELECT ?item ?itemLabel WHERE {\n ?item w...
2841,"""Can you find the rail link between Narvik, No...",LLMTranslator,,SELECT ?item ?itemLabel WHERE {\n ?item w...,,False,"""Can you find the rail link between Narvik, No...",SELECT ?station ?stationLabel ?cds ?line ?laye...,SELECT ?station ?stationLabel ?cds ?line ?laye...,"[{'item': {'type': 'uri', 'value': 'http://www...",SELECT ?item ?itemLabel WHERE {\n ?item w...
2842,"""Can you retrieve a list of fictional swords, ...",LLMTranslator,,SELECT ?item ?itemLabel WHERE {\n ?item w...,,False,"""Can you retrieve a list of fictional swords, ...",SELECT ?item ?itemLabel ?ownerLabel ?workLabel...,SELECT ?item ?itemLabel ?ownerLabel ?workLabel...,"[{'item': {'type': 'uri', 'value': 'http://www...",SELECT ?item ?itemLabel WHERE {\n ?item w...
2843,"""Can you write a SparQL query to count the num...",LLMTranslator,,SELECT ?item ?itemLabel WHERE {\n ?item w...,,False,"""Can you write a SparQL query to count the num...",SELECT ?partof (COUNT(*) as ?count)\n{\n?item ...,SELECT ?partof (COUNT(*) as ?count)\n{\n?item ...,"[{'item': {'type': 'uri', 'value': 'http://www...",SELECT ?item ?itemLabel WHERE {\n ?item w...


In [13]:
print(df['status'].iloc[0])

Unexpected err=ValueError('If `eos_token_id` is defined, make sure that `pad_token_id` is defined.'), type(err)=<class 'ValueError'>
